LOADING DATA AND CLEANING IT

In [1]:
import pandas as pd
train_data = pd.read_csv("/content/train.csv", encoding='latin1')
test_data=pd.read_csv("/content/test.csv",encoding='latin1')
train_df=train_data[['text','sentiment']]
test_df=test_data[['text','sentiment']]






PERFORMING NLP PREPROCESSING

PERORMING ZERO PADDING ETC TO EQUAL THE LENGHTH OF EACH SENTENCE IN VECTOR ORM FOR BETTER PERFORMANCE

In [2]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential


# Fill NaN values with an empty string
train_df['text'] = train_df['text'].fillna('')
test_df['text'] = test_df['text'].fillna('')

# Convert all text data to strings (in case some data are not string)
train_df['text'] = train_df['text'].astype(str)
test_df['text'] = test_df['text'].astype(str)

word_limit=10000
tokenizer=Tokenizer(num_words=word_limit)
tokenizer.fit_on_texts(train_df['text'])
print(tokenizer.word_counts)
print(tokenizer.word_index)

train_seq=tokenizer.texts_to_sequences(train_df['text'])
test_seq=tokenizer.texts_to_sequences(test_df['text'])
seq_length=max([len(a) for a in train_seq])
train_padding = pad_sequences(train_seq, maxlen=seq_length, padding='post')
test_padding=pad_sequences(test_seq,maxlen=seq_length, padding='post')





<ipython-input-2-9bfda41ef497>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['text'] = train_df['text'].fillna('')
<ipython-input-2-9bfda41ef497>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['text'] = test_df['text'].fillna('')
<ipython-input-2-9bfda41ef497>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

OrderedDict([('i', 16640), ('d', 311), ('have', 2600), ('responded', 1), ('if', 782), ('were', 355), ('going', 1103), ('sooo', 113), ('sad', 488), ('will', 910), ('miss', 618), ('you', 5330), ('here', 660), ('in', 3818), ('san', 20), ('diego', 11), ('my', 5543), ('boss', 22), ('is', 4011), ('bullying', 1), ('me', 2867), ('what', 1015), ('interview', 27), ('leave', 140), ('alone', 88), ('sons', 5), ('of', 3158), ('why', 491), ('couldn', 94), ('t', 3148), ('they', 784), ('put', 152), ('them', 433), ('on', 2870), ('the', 9018), ('releases', 2), ('we', 1032), ('already', 244), ('bought', 73), ('http', 1247), ('www', 122), ('dothebouncy', 1), ('com', 896), ('smf', 1), ('some', 790), ('shameless', 1), ('plugging', 3), ('for', 3656), ('best', 312), ('rangers', 3), ('forum', 9), ('earth', 23), ('2am', 7), ('feedings', 1), ('baby', 166), ('are', 1372), ('fun', 541), ('when', 685), ('he', 660), ('all', 1659), ('smiles', 11), ('and', 5164), ('coos', 1), ('soooo', 73), ('high', 73), ('both', 115),

In [3]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

all_sentiments = pd.concat([train_df['sentiment'], test_df['sentiment']], axis=0)

label_encoder = LabelEncoder()
label_encoder.fit(all_sentiments)

train_labels_encoded = label_encoder.transform(train_df['sentiment'])
test_labels_encoded = label_encoder.transform(test_df['sentiment'])

# Convert to one-hot encoding
train_labels_categorical = to_categorical(train_labels_encoded)
test_labels_categorical = to_categorical(test_labels_encoded)

# Check number of classes and shapes
print("Number of classes:", len(label_encoder.classes_))
print("Train Labels Shape:", train_labels_categorical.shape)
print("Test Labels Shape:", test_labels_categorical.shape)

Number of classes: 4
Train Labels Shape: (27481, 3)
Test Labels Shape: (4815, 4)


In [ ]:
print(test_labels_categorical)

[[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


LSTM MODEL

In [4]:
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,BatchNormalization
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.regularizers import l2
num_classes = train_labels_categorical.shape[1]

# Define the model
sequential_model = Sequential()
sequential_model.add(Embedding(26599, output_dim=150, input_length=seq_length))

# Add LSTM layers with Dropout and BatchNormalization
sequential_model.add(LSTM(units=128, return_sequences=True, kernel_regularizer=l2(0.01)))
sequential_model.add(Dropout(0.5))
sequential_model.add(BatchNormalization())

sequential_model.add(LSTM(units=128, return_sequences=False, kernel_regularizer=l2(0.01)))
sequential_model.add(Dropout(0.5))
sequential_model.add(BatchNormalization())

# Adjust the output layer to match the number of classes
sequential_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
sequential_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

training = sequential_model.fit(train_padding, train_labels_categorical, epochs=10,validation_split=0.2, verbose=1)




Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


687/687 ━━━━━━━━━━━━━━━━━━━━ 114s 157ms/step - accuracy: 0.3538 - loss: 2.9738 - val_accuracy: 0.4779 - val_loss: 1.1876
Epoch 2/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 106s 155ms/step - accuracy: 0.6211 - loss: 0.9212 - val_accuracy: 0.5612 - val_loss: 0.9951
Epoch 3/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 106s 154ms/step - accuracy: 0.7257 - loss: 0.6980 - val_accuracy: 0.6383 - val_loss: 0.8243
Epoch 4/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 140s 152ms/step - accuracy: 0.7600 - loss: 0.6325 - val_accuracy: 0.6538 - val_loss: 0.8512
Epoch 5/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 109s 159ms/step - accuracy: 0.7869 - loss: 0.5899 - val_accuracy: 0.6647 - val_loss: 0.8461
Epoch 6/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 142s 159ms/step - accuracy: 0.8078 - loss: 0.5476 - val_accuracy: 0.6638 - val_loss: 0.8478
Epoch 7/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 107s 156ms/step - accuracy: 0.8306 - loss: 0.5076 - val_accuracy: 0.6880 - val_loss: 0.8259
Epoch 8/10
687/687 ━━━━━━━━━━━━━━━━━━━━ 143s 158ms/step - accuracy: 0.8503 - loss: 0.46

In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict on the test data
test_predictions = sequential_model.predict(test_padding)
test_predictions_classes = np.argmax(test_predictions, axis=1)
test_labels_classes = np.argmax(test_labels_categorical, axis=1)

# Calculate metrics
accuracy = accuracy_score(test_labels_classes, test_predictions_classes)
precision = precision_score(test_labels_classes, test_predictions_classes, average='weighted')
recall = recall_score(test_labels_classes, test_predictions_classes, average='weighted')
f1 = f1_score(test_labels_classes, test_predictions_classes, average='weighted')

# Print the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


151/151 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step
Accuracy: 0.4897
Precision: 0.4307
Recall: 0.4897
F1 Score: 0.4353


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
def predict_sentiment(text, tokenizer, model, label_encoder, seq_length):
    ""

    # Step 1: Preprocess the input text
    text_seq = tokenizer.texts_to_sequences([text])
    text_padding = pad_sequences(text_seq, maxlen=seq_length, padding='post')

    # Step 2: Predict sentiment using the trained model
    pred_prob = sequential_model.predict(text_padding)
    pred_class = pred_prob.argmax(axis=1)

    # Step 3: Convert the prediction to a label
    predicted_label = label_encoder.inverse_transform(pred_class)

    return predicted_label[0]

# Example usage:
input_text = "I think this product is ok!"
predicted_sentiment = predict_sentiment(input_text, tokenizer, sequential_model, label_encoder, seq_length)
print(f"The predicted sentiment is: {predicted_sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
The predicted sentiment is: neutral
